# LVV-T297: Absolute Astrometric Performance

**Written By: Bryce Kalmbach**

**Last updated: 07-18-2019**

**Tested on Stack Version: w_2019_28**

## Requirements:

[OSS-REQ-0388](https://docushare.lsst.org/docushare/dsweb/Get/LSE-030#page=68)

Median error in absolute position for each axis, RA and DEC, shall be less than 50 milliarcseconds.

## Proposed Test Case:

1. Take images from region overlapping the Gaia footprint.  Repeat at multiple airmasses.

2. Perform source detection and astrometric measurement on images from step 1

3. Cross-match catalog from step 2 with Gaia catalog.  Select sources that are consistent with zero proper motion (according to Gaia).

4. Verify that the median error of the LSST positions (relative to the Gaia positions) is **50 milliarcseconds in RA, Dec independently**

### Import necessary tools

In [ ]:
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
from lsst.daf.persistence import Butler
import lsst.daf.persistence as daf_persistence

from astropy.coordinates import SkyCoord
from astropy import units as u

In [ ]:
# Make our plots nice and readable
plt.rcParams.update({'font.size': 18})

### Identify HSC Data to use

We want to get data from a single visit for this requirement so we choose a visit from the HSC Wide dataset. https://hsc-release.mtk.nao.ac.jp/doc/index.php/database/ has info 
on which tracts are included in the Wide data. We randomly choose tract 9348 for testing. To choose a different band for testing change `band` below.

In [ ]:
# Load a butler for the HSC Wide data
depth = 'WIDE'
band = 'HSC-R'
butler = daf_persistence.Butler('/datasets/hsc/repo/rerun/DM-13666/%s/'%(depth))

In [ ]:
# Find a visit in the WIDE data for specified band in the tract 9348
warp_list = os.listdir('/datasets/hsc/repo/rerun/DM-13666/WIDE/deepCoadd/%s/9348/0,0' % band)
warp_list.sort()
visit = int(warp_list[0].split('-')[-1].split('.')[0])

In [ ]:
subset = butler.subset('src', filter=band, visit=visit)

In [ ]:
# Load in sources from visit making exceptions for bad ccd 9 and focusing ccds.
hsc_sources_df = None
ccd_lims = []
for dataId in subset.cache:
    if dataId['ccd'] % 10 == 0:
        print('On CCD #%i' % dataId['ccd'])
    try:
        src_cat = butler.get('src', dataId=dataId)
        src_cat_df = src_cat.asAstropy().to_pandas()
        if hsc_sources_df is None:
            hsc_sources_df = pd.DataFrame([], columns=src_cat_df.columns)
            hsc_sources_df = hsc_sources_df.append(src_cat_df, sort=False)
        else:
            hsc_sources_df = hsc_sources_df.append(src_cat_df, sort=False)
        ccd_lims.append([np.degrees(np.max(src_cat_df['coord_ra'])),
                         np.degrees(np.min(src_cat_df['coord_ra'])),
                         np.degrees(np.max(src_cat_df['coord_dec'])),
                         np.degrees(np.min(src_cat_df['coord_dec']))])
    except daf_persistence.butlerExceptions.NoResults as inst:
        print('No results for CCD #%i' % dataId['ccd'])

In [ ]:
# Total number of HSC Sources
len(hsc_sources_df)

In [ ]:
hsc_sources_coords = SkyCoord(hsc_sources_df['coord_ra']*u.rad, hsc_sources_df['coord_dec']*u.rad)

In [ ]:
fig = plt.figure(figsize=(8, 8))
plt.scatter(hsc_sources_coords.ra.deg, hsc_sources_coords.dec.deg, s=8, lw=0)
plt.xlabel('RA (deg)')
plt.ylabel('dec (deg)')
plt.title('HSC Sources in Visit %i' %visit)
ax = plt.gca()
ax.set_xticks(ax.get_xticks()[1::2]) # Clean up ticks in RA

### Load Gaia data

We have previously created a pandas dataframe with Gaia data that overlaps the HSC Wide data footprint. Here we load it in and select the data in the region of the visit.

In [ ]:
# Load in cached gaia data
gaia_df = pd.read_pickle('/project/danielsf/gaia_hsc_overlap_pandas.pickle')

In [ ]:
gaia_df.head()

In [ ]:
# Only select data that falls in the bounds of the HSC CCDs
gaia_visit_df = pd.DataFrame([], columns=gaia_df.columns)
for ccd_corners in ccd_lims:
    gaia_visit_df = gaia_visit_df.append(gaia_df.query('ra < %f and ra > %f and dec < %f and dec > %f' % (ccd_corners[0], ccd_corners[1],
                                                                                                          ccd_corners[2], ccd_corners[3])))

In [ ]:
# Pick low proper motion sources (gaia proper motions are in mas/year)
low_pm = np.where((gaia_visit_df['pmra'] > -5) & (gaia_visit_df['pmra'] < 5) &
                  (gaia_visit_df['pmdec'] > -5) & (gaia_visit_df['pmdec'] < 5))[0]
gaia_visit_df = gaia_visit_df.iloc[low_pm]

In [ ]:
gaia_coords = SkyCoord(gaia_visit_df['ra']*u.deg, gaia_visit_df['dec']*u.deg)

In [ ]:
fig = plt.figure(figsize=(8, 8))
plt.scatter(gaia_coords.ra.deg, gaia_coords.dec.deg, s=8, lw=0)
plt.xlabel('RA (deg)')
plt.ylabel('dec (deg)')
plt.title('Gaia Sources in Visit %i' %visit)
ax = plt.gca()
ax.set_xticks(ax.get_xticks()[1::2]) # Clean up ticks in RA

### Use astropy to match

We will use the `match_to_catalog_sky` method from astropy to do the catalog match.

In [ ]:
idx, sep2d, sep3d = gaia_coords.match_to_catalog_sky(hsc_sources_coords)

In [ ]:
fig = plt.figure(figsize=(10,8))
plt.scatter(gaia_coords.ra.deg, gaia_coords.dec.deg, c=sep2d.arcsec*1000, s=20, vmax=50)
cb = plt.colorbar()
plt.xlabel('RA (deg)')
plt.ylabel('dec (deg)')
cb.set_label('Distance to match (milliarcsec)')
ax = plt.gca()
ax.set_xticks(ax.get_xticks()[1::2]) # Clean up ticks in RA

### Plot results against requirements

In [ ]:
# Look at overall separation
fig = plt.figure(figsize=(10, 8))
n, bins, _ = plt.hist(sep2d.arcsec, label='Gaia Objects', range=(0, 0.1))
plt.axvline(np.median(sep2d.arcsec), 0, np.max(n), c='r', lw=4, label='Median Separation = %.2f milliarcsec' % (1000*np.median(sep2d.arcsec)))
plt.title('Test of Absolute Astrometry')
plt.xlabel('Distance to match (arcsec)')
plt.ylabel('Number of Gaia Objects in Visit')
plt.legend()

In [ ]:
# Requirement specifies looking at RA and dec inpendently
sep_ra = gaia_coords.ra.arcsec - hsc_sources_coords.ra.arcsec[idx]
sep_dec = gaia_coords.dec.arcsec - hsc_sources_coords.dec.arcsec[idx]
ra_median_sep = np.median(np.abs(sep_ra))
dec_median_sep = np.median(np.abs(sep_dec))

In [ ]:
# Requirement specifies RA, dec individually so we look at those here
fig = plt.figure(figsize=(20, 8))

fig.add_subplot(1,2,1)
n, bins, _ = plt.hist(np.abs(sep_ra), label='Gaia Objects', range=(0, 0.1))
plt.axvline(ra_median_sep, 0, np.max(n), c='r', lw=4, label='Median Separation = %.2f milliarcsec' % (1000*ra_median_sep))
plt.axvline(.05, 0, np.max(n), label='Requirement =  50 milliarcsec', c='k', lw=4)
plt.title('Test of Absolute Astrometry RA')
plt.xlabel('Distance to match (arcsec)')
plt.ylabel('Number of Gaia Objects in Visit')
plt.legend()

fig.add_subplot(1,2,2)
n, bins, _ = plt.hist(np.abs(sep_dec), label='Gaia Objects', range=(0, 0.1))
plt.axvline(dec_median_sep, 0, np.max(n), c='r', lw=4, label='Median Separation = %.2f milliarcsec' % (1000*dec_median_sep))
plt.axvline(0.05, 0, np.max(n), label='Requirement =  50 milliarcsec', c='k', lw=4)
plt.title('Test of Absolute Astrometry dec')
plt.xlabel('Distance to match (arcsec)')
plt.ylabel('Number of Gaia Objects in Visit')
plt.legend()

The requirements are satisfied if both RA and dec median values are less than 50 milliarcseconds.

### Test against requirements

If these fail with a new version of the stack our CI testing of notebooks will also fail and alert us.

In [ ]:
class RequirementFailure(ValueError):
    "Requirement not met."

In [ ]:
# Set up for potential error messages
error_msg = ""
error_present = False
error_val = 0

In [ ]:
# Test RA. Convert to milliarcsec by multiplying by 1000.
if ra_median_sep*1000. > 50:
    error_present = True
    error_val += 1
    error_msg += str('Error #%i: \n' % error_val + 
                     'Median RA Astrometry Error greater than 50 milliarcsec. Test Value: %.2f mas \n' % (ra_median_sep*1000.))

In [ ]:
# Test dec. Convert to milliarcsec by multiplying by 1000.
if dec_median_sep*1000. > 50:
    error_present = True
    error_val += 1
    error_msg += str('Error #%i: \n' % error_val +
                     'Median dec Astrometry Error greater than 50 milliarcsec. Test Value: %.2f mas' % (dec_median_sep*1000.))

In [ ]:
if error_present is True:
    error_msg = str('%i Total Errors: \n' % error_val + error_msg)
    raise RequirementFailure(error_msg)